## Libraries Used

In [1]:
import datetime
import os

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *


### Read the Main Table

In [2]:
# Read from main_table
Table = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main table.csv")

# Add record type to main table
Table_new = Table.withColumn('record_type', lit("A"))

# Write in main_table folder
path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
try:
    Table_new.write.format('csv').save(
        os.path.join(path, 'main_table'), header=True)
except:
    Table_new.write.mode('overwrite').format('csv').save(
        os.path.join(path, 'main_table'), header=True)


In [3]:
# GFG
class my_dictionary(dict):

    # __init__ function
    def __init__(self):
        self = dict()

    # Function to add key:value
    def add(self, key, value):
        self[key] = value

In [4]:
# data_schema = [
#         StructField('p_id', IntegerType(), True),
#         StructField('p_name', StringType(), True),
#         StructField('price', IntegerType(), True),
#         StructField('date_timestamp', TimestampType(), True),
#         StructField('record_type', StringType(), True)
#     ]
# final_struc = StructType(fields=data_schema)
# path1 = os.path.join(
#             "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
# try: 
#     os.remove(os.path.join(path1,'_SUCCESS'))
# except :
#     pass
# # mainTable = spark.read.format('csv').options(
# #             header=True).load(path, schema=final_struc)
# mainTable = spark.read.csv(path1,header=True,schema=final_struc)
# print(path1)
# mainTable.show()

In [5]:
total_products_updated = my_dictionary()
total_products_inserted = my_dictionary()
total_products_deleted = my_dictionary()


for i in range(10, 0, -1):

    data_schema = [
        StructField('p_id', IntegerType(), True),
        StructField('p_name', StringType(), True),
        StructField('price', IntegerType(), True),
        StructField('date_timestamp', TimestampType(), True),
        StructField('record_type', StringType(), True)
        ]
    # Schema which we are accepting
    final_struc = StructType(fields=data_schema)

    try:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", t)
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass

        mainTable = spark.read.csv(path1, header=True, schema=final_struc)
    except:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass
        mainTable = spark.read.csv(path1, header=True, schema=final_struc)

    print("\n File read from ::")
    print(path1, end='\n')

    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = (datetime.datetime.today() - datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    print("\n\t\ Date ----> "+previous_day+"\n")

    # before_insert = str(mainTable.count())
    print("Main_Table Count ----> " + str(mainTable.count()))

    # Address to the Previous Day folder
    new_address = address + previous_day
#     print("\nNew Address to read the folder ---->"+new_address)

    # Read the Previous Day folder
    per_day_data=spark.read.format('csv') \
        .options(header=True, inferschema=True) \
        .load(new_address)

    # Get the list of p_id which we have to inserted
    to_be_inserted=per_day_data.filter("record_type == 'I' ").collect()
    p_id_list_I=[i.p_id for i in to_be_inserted]

    # Directly append new Inserted products
    mainTable_I_inserted=mainTable.union(per_day_data.filter("record_type == 'I' "))


    after_insert=str(mainTable_I_inserted.count())
    total_insert=str(per_day_data.filter("record_type == 'I' ").count())

    print("After Inserting----> " + after_insert)
    print("Total \"Inserted I\"----> " + total_insert)
    
    total_products_inserted.add(previous_day, p_id_list_I)
    print(" ".join(map(str, p_id_list_I)))
    print("\n")

    # Created a new DataFrame of records to be updated
    from_per_day_data_U=per_day_data.filter("record_type == 'U' ")

    from_per_day_data_U_list=from_per_day_data_U.select("p_id").collect()

#     print("Output of our collect operation----> ",from_per_day_data_U_list[0])
#     print("get the value of p_id -----> ",from_per_day_data_U_list[0].p_id)

# List comprehension
    p_id_list_U=[i.p_id for i in from_per_day_data_U_list]
    print("TO Be Update\"per_day_data\"")
    print(" ".join(map(str, p_id_list_U)))
#     print(p_id_list_U)

    total_products_updated.add(previous_day, p_id_list_U)

    total_update=str(len((p_id_list_U)))
    print("Update Count ------>"+total_update)
    print("\n")

    from_mainTable_U=mainTable_I_inserted.filter(
        col('p_id').isin(p_id_list_U))

    # Performed Union operation on the above tables
    mT_and_pDD_union=from_mainTable_U.union(from_per_day_data_U)
#     mT_and_pDD_union.orderBy(mT_and_pDD_union.p_id).show()

    # Performed GroupBy operation on P_ID and took the latest date
    for_join_mT_and_pDD=mT_and_pDD_union.groupBy("p_id").agg(
        {"date_timestamp": "max"}).withColumnRenamed("max(date_timestamp)", "date_timestamp_1")
    for_join_mT_and_pDD=for_join_mT_and_pDD.withColumnRenamed(
        "p_id", "p_id_1")
#     for_join_mT_and_pDD.show()

    # Performed Join opteration to pick only latest updates only
    joined=mT_and_pDD_union.join(for_join_mT_and_pDD, (
        mT_and_pDD_union.p_id == for_join_mT_and_pDD.p_id_1) & (
            mT_and_pDD_union.date_timestamp == for_join_mT_and_pDD.date_timestamp_1), 'inner')

    joined=joined.select(
        ['p_id', 'p_name', 'price', 'date_timestamp', 'record_type'])
#     joined.count()

    # First remove the p_id from Main_Table which we have to update
    mainTable_U_updated=mainTable_I_inserted.filter(
        ~col('p_id').isin(p_id_list_U))
#     mainTable_U_updated.orderBy("p_id").count()

    # Then Add the Updated P_ID to the Main_Table
    mainTable_U_updated_new=mainTable_U_updated.union(joined)
#     after_update = str(mainTable_U_updated_new.orderBy("p_id").count())
#     mainTable_U_updated_new.orderBy("p_id").count()

    # Get the list of p_id which we have to delete
    to_be_deleted=per_day_data.filter("record_type == 'D' ").collect()
    p_id_list_D=[i.p_id for i in to_be_deleted]

    total_products_deleted.add(previous_day, p_id_list_D)

#     print("\nList of p_id which we have to deleted taken from \"per_day_data\"")
#     print(p_id_list_D)
    total_deleted=str(len((p_id_list_D)))
    print("\nTotal Deleted ------>"+str(len((p_id_list_D))))
    print(" ".join(map(str, p_id_list_D)))

    # Remove the deleted p_id from main_table
    mainTable_D_deleted=mainTable_U_updated_new.filter(
        ~col('p_id').isin(p_id_list_D))

    after_delete=str(mainTable_D_deleted.count())
    print("After Deleting Count---->"+after_delete)



    # This writes the DF in different files becaues of parallism
    t="main_table"+str(previous_day)
#     print(t,end="\n")

#     try :
    mainTable_D_deleted.write.format('csv').save(os.path.join(path, t),header=True)
    print("\n Main Table Stored")
    print(os.path.join(path, t))
#     except :
#         mainTable_D_deleted.write.mode('overwrite').format('csv').save(os.path.join(path, 'main_table'))

    print("\n***************************************************************************************\n")


 File read from ::
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table

	\ Date ----> 29-03-2020

Main_Table Count ----> 200
After Inserting----> 205
Total "Inserted I"----> 5
201 225 205 215 201


TO Be Update"per_day_data"
16 28 21 120 32 9 145 50 38 62 179 174 146
Update Count ------>13



Total Deleted ------>5
47 82 99 187 172
After Deleting Count---->200

 Main Table Stored
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table29-03-2020

***************************************************************************************


 File read from ::
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table29-03-2020

	\ Date ----> 30-03-2020

Main_Table Count ----> 200
After Inserting----> 208
Total "Inserted I"----> 8
240 222 219 233 224 207 206 207


TO Be Update"per_day_data"
153 165 57 7 173 127 75 32 115 116 157 121 118 160 83 142 199 188 66 5 107 156 9
Update Count ------>23



Total Deleted ------>10
6 184 20 48 63 93 28 90 1

In [6]:
import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)

for i in range(10,0,-1) :
    previous_day = (datetime.datetime.today() - datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    pp.pprint(previous_day)
    print("inserted :\t")
    print(' '.join(map(str, total_products_inserted.get(previous_day))))
#     pp.pprint(total_products_inserted.get(previous_day))
    print("updated :\t")
    print(' '.join(map(str, total_products_updated.get(previous_day))))
#     pp.pprint(total_products_updated.get(previous_day))
    print("deleted :\t")
    print(' '.join(map(str, total_products_deleted.get(previous_day))))
#     pp.pprint(total_products_deleted.get(previous_day))
    print("\n")

'29-03-2020'
inserted :	
201 225 205 215 201
updated :	
16 28 21 120 32 9 145 50 38 62 179 174 146
deleted :	
47 82 99 187 172


'30-03-2020'
inserted :	
240 222 219 233 224 207 206 207
updated :	
153 165 57 7 173 127 75 32 115 116 157 121 118 160 83 142 199 188 66 5 107 156 9
deleted :	
6 184 20 48 63 93 28 90 142 133


'31-03-2020'
inserted :	
227 210 242 243 206 223 203 211 209 238 220 210 215
updated :	
147 84 15 51 24 145 116 86 161
deleted :	
196 61 83 138 164


'01-04-2020'
inserted :	
235 221 218 213 202 216 201 234 214 231 236 210
updated :	
189 156 145 55 36 35 55 59 3 57 113 137 71 2 52
deleted :	
195


'02-04-2020'
inserted :	
227 209 203 247 229 235 248 237 247 228 235
updated :	
12 65 150 71 104 191 182 160 80 94 101 49 115 123
deleted :	
91 127 141 166 150 56 152 177


'03-04-2020'
inserted :	
239 230 213 204 222 250 249 210
updated :	
54 58 74 78 71 160 139 173 26 34 69 55 157 26 117 78
deleted :	
112 160 88 158 198 32 137 171


'04-04-2020'
inserted :	
228 216 222 239 